Длинные названия потому, что мне лень писать документацию, а за жизненным циклом обектов следить не умею :)

In [1]:
Generate_Network_t0 <- function(size = 40, sparsity = 0.7) {
  network <- matrix(0, nrow=size, ncol=size)
  
  for (i in 1:size) {
    for (j in 1:size) {
      if (runif(1) > sparsity) {
        network[i, j] <- 1
      }
    }
  }
  
  return(network)
}


# Смулирую ТОЛЬКО возникновение сети


Network_function_1 <- function(i, Network) { # Зачем тут минишаг? Для ковариат сложнее?
  net <- 0
  for (j in 1:length(Network[i, ])) {
    net <- net + Network[i, j]
  }
  return(net)
}

Generate_Possible_Ministep_Matrix_vector <- function(i, Network) {
  Networks <- list()
  
  for (j in 1:length(Network[i, ])) {
    if (i != j) {
      TMP_Net <- Network
      TMP_Net[i, j] <- 1
      Networks[[j]] <- TMP_Net    
    }
  }
  return(Networks)
}

Ministep <- function(Network, i, j) {
  Network[i, j] <- 1
  
  return(Network)
}

Distribution_actors <- function(i, Network, beta) {
  Util_i_j <- 0
  Util <- 0
  for (j in 1:length(Network[i, ])) {

        Util_i_j[j] <- Utility(Network, beta, i, j)

        Util_i_h <- 0
        for (h in 1:length(Network[i, ])) {
          if (j != h) {
            Util_i_h <- Util_i_h + Utility(Network, beta, i, h)
          }
        }
        
        Util[j] <- Util_i_j[j] / Util_i_h
      
        Util[i] <- 0

  }
  return(Util)
}


Utility <- function(Network, beta, i, j) {
  util <- 0
  util <- util + (beta * Network_function_1(i, Network)) # Если есть еще функции добавлять через сумму
  
  return(util)
}


Simulator <- function(Network0, t = 0, LambdaV, total_Lambda, T, beta) {
  Network <- Network0 
  while (t < T) {
    deltaT <- rexp(1, rate=total_Lambda)
    i <- sample(1:length(Network0[1, ]), size=1, prob=(LambdaV / total_Lambda))
    j <- sample(1:length(Network0[, 1]), size=1, prob=Distribution_actors(i, Network, beta)) # Тут надо задать распределение
    Network[i, j] <- 1
    t <- t + deltaT
  }
  return(Network)
}


In [2]:
size <- 40
Network0 <- Generate_Network_t0(size = size)
LambdaV <- runif(n = size, min = 0.1, max = 5) 
total_Lambda <- sum(LambdaV)
beta <- .6
T <- 2

In [3]:
write.table(Network0, file = "Network0.dat", sep = " ", row.names = FALSE, col.names = FALSE)

In [4]:
Network1 <- Simulator(Network0, t = 0, LambdaV, total_Lambda, T, beta)

In [5]:
write.table(Network1, file = "Network1.dat", sep = " ", row.names = FALSE, col.names = FALSE)

In [6]:
Network2 <- Simulator(Network1, t = 0, LambdaV, total_Lambda, T, beta)

In [7]:
write.table(Network2, file = "Network2.dat", sep = " ", row.names = FALSE, col.names = FALSE)

In [8]:
sum(Network0)
sum(Network1)
sum(Network2)

[1] 484

[1] 593

[1] 685